# How to deal with complex/large Documents

In the previous notebook, we developed a solution for various types of files and data formats commonly found in organizations, and this covers big majority of the use cases. However, you will find that there are issues when dealing with questions that require answers from complex files. The complexity of these files arises from their length and the way information is distributed within them. Large documents are always a challenge for Search Engines.

One example of such complex files is Technical Specification Guides or Product Manuals, which can span hundreds of pages and contain information in the form of images, tables, forms, and more. Books are also complex due to their length and the presence of images or tables.

These files are typically in PDF format. To better handle these PDFs, we need a smarter parsing method that treats each document as a special source and processes them page by page (1 page = 1 chunk). The objective is to obtain more accurate and faster answers from our system. Fortunately, there are usually not many of these types of documents in an organization, allowing us to make exceptions and treat them differently.

If your use case is just PDFs, for example, you can just use [PyPDF library](https://pypi.org/project/pypdf/) or [Azure AI Document Intelligence SDK (former Form Recognizer)](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-3.0.0), vectorize using OpenAI API and push the content to a vector-based index. And this is problably the simplest and fastest way to go.  However if your use case entails connecting to a datalake, or Sharepoint libraries or any other document data source with thousands of documents with multiple file types and that can change dynamically, then you would want to use the Ingestion and Document Cracking and AI-Enrichment capabilities of Azure Search engine, Notebooks 1-2, and avoid a lot of painful custom code. 


In [1]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
from typing import List

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


from common.utils import parse_pdf, read_pdf_files, text_to_base64
from common.prompts import DOCSEARCH_PROMPT
from common.utils import CustomAzureSearchRetriever


from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))
    
os.makedirs("data/books/",exist_ok=True)
 
BLOB_CONTAINER_NAME = "books"

c = os.environ["BLOB_CONNECTION_STRING"].split(";")
BASE_CONTAINER_URL = c[0].split("=")[1] + BLOB_CONTAINER_NAME + "/"

LOCAL_FOLDER = "./data/books/"

os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=1) 

## 1 - Manual Document Cracking with Push to Vector-based Index

Within our demo storage account, we have a container named `books`, which holds 5 books of different lengths, languages, and complexities. Let's create a `cogsrch-index-books-vector` and load it with the pages of all these books.

We begin by downloading these books to our local machine:

In [ ]:
# books = ["Azure_Cognitive_Search_Documentation.pdf", 
#          "Boundaries_When_to_Say_Yes_How_to_Say_No_to_Take_Control_of_Your_Life.pdf",
#          "Fundamentals_of_Physics_Textbook.pdf",
#          "Made_To_Stick.pdf",
#          "Pere_Riche_Pere_Pauvre.pdf"]

In [4]:
books = ["azure-ai-studio.pdf", "azure-ai-services.pdf"]

Let's download the files to the local `./data/` folder:

In [5]:
for book in tqdm(books):
    book_url = BASE_CONTAINER_URL + book + os.environ['BLOB_SAS_TOKEN']
    urllib.request.urlretrieve(book_url, LOCAL_FOLDER+ book)

  0%|          | 0/2 [00:00<?, ?it/s]


HTTPError: HTTP Error 404: The specified blob does not exist.

### What to use: pyPDF or AI Documment Intelligence API (Form Recognizer)?

In `utils.py` there is a **parse_pdf()** function. This utility function can parse local files using PyPDF library and can also parse local or from_url PDFs files using Azure AI Document Intelligence (Former Form Recognizer).

If `form_recognizer=False`, the function will parse the PDF using the python pyPDF library, which 75% of the time does a good job.<br>

Setting `form_recognizer=True`, is the best (and slower) parsing method using AI Documment Intelligence API (former known as Form Recognizer). You can specify the prebuilt model to use, the default is `model="prebuilt-document"`. However, if you have a complex document with tables, charts and figures , you can try
`model="prebuilt-layout"`, and it will capture all of the nuances of each page (it takes longer of course).

**Note: Many PDFs are scanned images. For example, any signed contract that was scanned and saved as PDF will NOT be parsed by pyPDF. Only AI Documment Intelligence API will work.**

In [6]:
book_pages_map = dict()
for book in books:
    print("Extracting Text from",book,"...")
    
    # Capture the start time
    start_time = time.time()
    
    # Parse the PDF
    book_path = LOCAL_FOLDER+book
    book_map = parse_pdf(file=book_path, form_recognizer=True, verbose=True)
    book_pages_map[book]= book_map
    
    # Capture the end time and Calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Parsing took: {elapsed_time:.6f} seconds")
    print(f"{book} contained {len(book_map)} pages\n")

Extracting Text from azure-ai-studio.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 178.611562 seconds
azure-ai-studio.pdf contained 654 pages

Extracting Text from azure-ai-services.pdf ...
Extracting text using Azure Document Intelligence
Parsing took: 64.384216 seconds
azure-ai-services.pdf contained 290 pages



Now let's check a random page of each book to make sure the parsing was done correctly:

In [7]:
for bookname,bookmap in book_pages_map.items():
    print(bookname,"\n","chunk text:",bookmap[random.randint(10, 50)][2][:120],"...\n")

azure-ai-studio.pdf 
 chunk text: 1. 아직 플레이그라운드에 있지 않다면 상단 메뉴에서 빌드를 선택한 다음 축소 가능 한 왼쪽 메뉴에서 플레이그라운드를 선택합니다.
2. 도우미 설정 창에서 데이터 추가(미리 보기)>+ 데이터 원본 추가를 선택합니다. ...

azure-ai-services.pdf 
 chunk text: Azure AI서비스에대한사용자지정하위 도메인 이름
아티클 • 2024. 02. 29.
2019년 7월부터 Azure AI 서비스는 Azure Portal , Azure Cloud Shell 또는 Azure CLI를 ...



As we can see above, all books were parsed except `Pere_Riche_Pere_Pauvre.pdf` (this book is "Rich Dad, Poor Dad" written in French), why? Well, as we mentioned above, this book was scanned, so each page is an image and with a very unique font. We need a good PDF parser with good OCR capabilities in order to extract the content of this PDF. 
Let's try to parse this book again, but this time using Azure Document Intelligence API (former Form Recognizer)

In [8]:
%%time
book = "azure-ai-services-openai.pdf"
book_path = LOCAL_FOLDER+book
book_map = parse_pdf(file=book_path, form_recognizer=True, model="prebuilt-document",from_url=False, verbose=True)
book_pages_map[book]= book_map

Extracting text using Azure Document Intelligence


KeyboardInterrupt: 

In [ ]:
print(book,"\n","chunk text:",book_map[random.randint(10, 50)][2][:80],"...\n")

As demonstrated above, Azure Document Intelligence proves to be superior to pyPDF. **For production scenarios, we strongly recommend using Azure Document Intelligence consistently**. When doing so, it's important to make a wise choice between the available models, such as "prebuilt-document," "prebuilt-layout," or others. You can find more information on model selection [HERE](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/choose-model-feature?view=doc-intel-3.0.0).


## Create Vector-based index


Now that we have the content of the book's chunks (each page of each book) in the dictionary `book_pages_map`, let's create the Vector index in our Azure Search Engine where this content is going to land

In [9]:
book_index_name = "cogsrch-index-books"

In [10]:
### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}


Please note the following points regarding the index:

- The ParentKey field is absent.
- The page_num field is present.

The absence of the ParentKey field is due to the utilization of a PUSH method, rather than a PULL method. This approach indicates that we are not leveraging the integrated indexing provided by the Azure AI Search engine. Instead, we are engaging in the process of parsing, performing OCR, and manually creating and pushing the content along with its vectors.

This manual parsing process involves the use of either, the pyPDF library, or the Azure Document Intelligence API. These APIs allow for the segmentation of content by page rather than by a specified number of characters, which is the method employed by the Azure AI search indexer. Consequently, this enables the inclusion of page_num as a field in our index.

REST API version 2023-10-01-Preview supports external and internal vectorization. This Notebook assumes an external vectorization strategy. This API also supports:
    
- vectorSearch algorithms, hnsw and exhaustiveKnn nearest neighbors, with parameters for indexing and scoring.
- vectorProfiles for multiple combinations of algorithm configurations.

Vector search algorithms include **exhaustive k-nearest neighbors (KNN)** and **Hierarchical Navigable Small World (HNSW)**. Exhaustive KNN performs a brute-force search that scans the entire vector space. HNSW performs an approximate nearest neighbor (ANN) search. While KNN provides exact nearest neighbor search results with high accuracy, its computational cost and poor scalability make it impractical for large datasets or real-time applications. HNSW, on the other hand, offers a highly efficient and scalable solution for nearest neighbor searches by finding approximate nearest neighbors quickly, making it more suitable for large-scale and high-dimensional data applications.


check [HERE](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index?tabs=config-2023-10-01-Preview%2Crest-2023-11-01%2Cpush%2Cportal-check-index) for the details of the vector configuration.

In [11]:
index_payload = {
    "name": book_index_name,
    "vectorSearch": {
        "algorithms": [  # We are showing here 3 types of search algorithms configurations that you can do
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-hnsw-config-2",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 8,
                     "efConstruction": 800,
                     "efSearch": 800,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-eknn-config",
                 "kind": "exhaustiveKnn",
                 "exhaustiveKnnParameters": {
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-2",
             "algorithm": "my-hnsw-config-2",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-3",
             "algorithm": "my-eknn-config",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "page_num","type": "Edm.Int32","searchable": "false","retrievable": "true"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,
            "vectorSearchProfile": "my-vector-profile-3", # we picked profile 3 to show that this index uses eKNN vs HNSW (on prior notebooks)
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
        
    ],
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [ ]:
# Uncomment to debug errors
# r.text

## Upload the Document chunks and its vectors to the Index

The following code will iterate over each chunk of each book and use the Azure Search Rest API upload method to insert each document with its corresponding vector (using OpenAI embedding model) to the index.

In [12]:
%%time
for bookname,bookmap in book_pages_map.items():
    print("Uploading chunks from",bookname)
    for page in tqdm(bookmap):
        try:
            page_num = page[0] + 1
            content = page[2]
            book_url = BASE_CONTAINER_URL + bookname
            upload_payload = {
                "value": [
                    {
                        "id": text_to_base64(bookname + str(page_num)),
                        "title": f"{bookname}_page_{str(page_num)}",
                        "chunk": content,
                        "chunkVector": embedder.embed_query(content if content!="" else "-------"),
                        "name": bookname,
                        "location": book_url,
                        "page_num": page_num,
                        "@search.action": "upload"
                    },
                ]
            }

            r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name + "/docs/index",
                                 data=json.dumps(upload_payload), headers=headers, params=params)
            if r.status_code != 200:
                print(r.status_code)
                print(r.text)
        except Exception as e:
            print("Exception:",e)
            print(content)
            continue

Uploading chunks from azure-ai-studio.pdf
Uploading chunks from azure-ai-services.pdf
CPU times: user 1min 27s, sys: 2.75 s, total: 1min 30s
Wall time: 6min 11s


100%|██████████| 290/290 [01:41<00:00,  2.86it/s]


## Query the Index

In [ ]:
#QUESTION = "What's Azure AI Search?"
QUESTION = "Can I move, backup, and restore indexes?"

In [ ]:
indexes = [book_index_name]
k=10 # in this index k corresponds to the top pages as well

In [ ]:
retriever = CustomAzureSearchRetriever(indexes=[book_index_name], topK=k, reranker_threshold=1)

**Note**: that we are picking a larger k=20 since these chunks are NOT of 5000 chars each like prior notebooks, but instead each page is a chunk.

In [ ]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS),
)

In [ ]:
In `utils.py` we created the **CustomAzureSearchRetriever** class that we will use going forward

In [ ]:
chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the 4 variables above to the prompt template
    | llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

#### With GPT 3.5

In [ ]:
for chunk in chain.with_config(configurable={"model": "gpt35"}).stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

#### With GPT 4

In [ ]:
for chunk in chain.with_config(configurable={"model": "gpt4"}).stream(
    {"question": QUESTION, "language": "English"}):
    print(chunk, end="", flush=True)

# Summary

In this notebook we learned how to deal with complex and large Documents and make them available for Q&A over them using [Hybrid Search](https://learn.microsoft.com/en-us/azure/search/search-get-started-vector#hybrid-search) (text + vector search).

We also learned the power of Azure Document Inteligence API and why it is recommended for production scenarios where manual Document parsing (instead of Azure Search Indexer Document Cracking) is necessary.

Using Azure AI Search with its Vector capabilities and hybrid search features eliminates the need for other vector databases such as Weaviate, Qdrant, Milvus, Pinecone, and so on.


# NEXT
So far we have learned how to use OpenAI vectors and completion APIs in order to get an excelent answer from our documents stored in Azure AI Search. This is the backbone for <font color="red">a GPT Smart Search Engine.</font>

However, we are missing something: **How to have a conversation with this engine?**

On the next Notebook, we are going to understand the concept of **memory**. This is necessary in order to have a chatbot that can establish a conversation with the user. Without memory, there is no real conversation.